#### 1. 모듈 가져오기

In [1]:
import pandas as pd
import pymysql
from sqlalchemy import create_engine

#### 2. 원본 파일 읽어오기

In [2]:
file = open("./books.txt", "r", encoding='UTF8')

books  = []
while True:
    strr = file.readlines()
    if not strr:
        break
    books += strr

books = books[40:]

#### 3. 변수 및 메소드 선언, 구분자 생성

In [3]:
def getTitle(string):
    return string.split(".")[1][1:-1]

In [4]:
# 데이터 프레임 생성
column_name = ["b_id", "title", "author", "content", "cnt"]
books_df = pd.DataFrame(columns=column_name)

# 동화 끝 구분자
stop_list = ["(끝)\n", "(동화 끝)\n"]

# 변수 선언
title_idx = 0
dict = {}
dict["content"] = ""
b_idx = -1

#### 4. 동화책 분류

In [5]:
while b_idx < len(books)-1:
    b_idx += 1
    if books[b_idx].split(".")[0] == str(title_idx+1) : # 제목 선정
        title_idx += 1
        dict = {}
        dict["b_id"] = title_idx
        dict["title"] = getTitle(books[b_idx])
        dict["content"] = ""
    elif books[b_idx] == "\n" : # 공백은 pass
        continue
    elif books[b_idx] in stop_list : # 내용 끝남
        df = pd.DataFrame(dict, index = [0])
        books_df = pd.concat([books_df,df])

        if title_idx >= 210 : # 모든 동화 끝
            break

        while b_idx<len(books): # 동화 끝나고 필요 없는 부분 제거
            b_idx += 1
            if books[b_idx].split(".")[0] == str(title_idx+1) :
                b_idx -= 1
                break
    else : # 내용 추가
        dict["content"] += books[b_idx]

# 인덱스 재설정
books_df = books_df.reset_index(drop=True)

# AR구현할 책을 상단으로
books_df.loc[52, "b_id"] = 2
books_df.loc[1, "b_id"] = 53
books_df.loc[2, "b_id"] = 211
books_df.loc[3, "b_id"] = 212

# 이 두가지 책은 아직 내용 없음
df = pd.DataFrame({"b_id":3, "title":"토끼와 거북이", "author":"이솝 우화", "content":"", "cnt":0}, index = [0])
books_df = pd.concat([books_df,df])
df = pd.DataFrame({"b_id":4, "title":"양치기 소년", "author":"이솝 우화", "content":"", "cnt":0}, index = [0])
books_df = pd.concat([books_df,df])

# 정렬 및 인덱스 재설정
books_df = books_df.sort_values("b_id")
books_df = books_df.reset_index(drop=True)

# 조회수, 저자명 채우기
books_df["cnt"] = 0
books_df["author"] = "그림 형제"

#### 5. 전처리 작업

In [6]:
page_separator = "&page&"
for i in range(len(books_df["b_id"])) :
    # 제목에서 괄호 없애기
    title = books_df.loc[i, "title"]
    if title.find("(") > 0 :
        books_df.loc[i, "title"] = title[:title.find("(")]
        
    # 내용에 들어간 번역가 첨언 제거
    content = books_df.loc[i, "content"]
    if content.find("(본문시작)") > 0 :
        content= content[content.find("(본문시작)")+10:]
    


In [ ]:
for i in books_df.loc[4].content.split(page_separator) :
    print(i)
    print("===============")

In [10]:
# 텍스트 최대 길이
reee = 0;
idx_list = books_df.index
for i in idx_list:
    if(reee < len(books_df.loc[i, "content"])) :
        reee = len(books_df.loc[i, "content"])
reee

26648

#### 6. DB에 저장

In [ ]:
db_connection_str = 'mysql+pymysql://[user]:[password]@[host]/[db]' # 로컬에 저장
db_connection = create_engine(db_connection_str)
conn = db_connection.connect()

books_df.to_sql(name='book', con=db_connection, if_exists='append',index=False)  